# Exploring the Security of One-Time and Many-Time Pads

## Overview
This team project investigates the cryptographic security of the one-time pad, known for its perfect security under certain conditions, and explores the implications of reusing a key, referred to as a many-time pad. We will examine the practical aspects of implementing these encryption methods, analyze their vulnerabilities, and develop strategies to exploit these vulnerabilities in a controlled environment. Here is some background story in signal intelligence. https://en.wikipedia.org/wiki/Venona_project
## Objectives
1. Understand One-Time Pad
Examine the concept of the one-time pad, its implementation, and why it is considered perfectly secure.
2. Implement One-Time Pad Encryption and Decryption
Develop a Python program that simulates the encryption and decryption process between two parties.
3. Explore Many-Time Pad
Extend the one-time pad implementation to simulate a many-time pad scenario, where a single key is used to encrypt multiple messages.
4. Cryptanalysis of Many-Time Pad
Design and execute an attack strategy to decrypt messages encrypted with a many-time pad, focusing on exploiting the vulnerabilities introduced by key reuse.

## Problem 1: Understanding One-Time Pad
Tasks:
1. Research the theoretical basis of the one-time pad, including its requirements and operational principles.

## Problem 2: One-Time Pad Implementation
Tasks: the encryption and decryption process between two parties, Alice and Bob.
1. Alice's Program
Should prompt for a message input (plaintext), then display the ciphertext, and save both the ciphertext (in hex) and the key (in hex) in separate files.
2. Bob's Program:
Should read the key and ciphertext from their respective files and display the decrypted plaintext.

In [18]:
from pathlib import Path
import random
import string

Path('./mailbox/').mkdir(exist_ok=True)
mailbox = Path('./mailbox/')
print(f'Using mailbox: {mailbox}')

def generate_key(length):
    return ''.join(random.choices(string.ascii_lowercase, k=length))

def vigenere_encrypt(text, key):
    encrypted_text = []
    n = len(text)
    for i in range(n):
        x = ord(text[i]) + ((ord(key[i]) % n) % 26)
        encrypted_text.append(chr(x))
    return "".join(encrypted_text)

def vigenere_decrypt(encrypted_text, key):
    decrypted_text = []
    n = len(encrypted_text)
    for i in range(n):
        x = ord(encrypted_text[i]) - ((ord(key[i]) % n) % 26)
        decrypted_text.append(chr(x))
    return "".join(decrypted_text)

def one_time_pad_encrypt(plaintext, key):
    return vigenere_encrypt(plaintext, key)

def one_time_pad_decrypt(ciphertext, key):
    return vigenere_decrypt(ciphertext, key)

def string_to_hex(s):
    return s.encode('utf-8').hex()

def hex_to_string(h):
    return bytes.fromhex(h).decode('utf-8')

def write_message(file_name, message):
    mail_file = mailbox / f'{file_name}.txt'
    with mail_file.open('w') as f:
        f.write(string_to_hex(message))

def read_message(file_name):
    mail_file = mailbox / f'{file_name}.txt'
    with mail_file.open() as f:
        return hex_to_string(f.read())

def alice_program(message_name):
    # get user input
    plaintext = input('Enter a message: ')
    print(f'Plaintext : {plaintext}')
    # encrypt the message
    key = generate_key(len(plaintext))
    ciphertext = one_time_pad_encrypt(plaintext, key)
    print(f'Ciphertext: {ciphertext}')
    write_message(message_name, ciphertext)
    write_message(message_name+'_key', key)

def bob_program(message_name):
    ciphertext = read_message(message_name)
    key = read_message(message_name+'_key')
    message = one_time_pad_decrypt(ciphertext, key)
    print(f'Plaintext : {message}')

message_file_name = 'message1'
alice_program(message_file_name)
bob_program(message_file_name)

Using mailbox: mailbox
Plaintext : Hello
Ciphertext: Ieolp
Plaintext : Hello
